In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
import moola
from SimpleBrainMesh import generate_doughnut_mesh
from BiotSolver import solve_biot
from fenics_adjoint import *
from pyadjoint import ipopt
set_log_level(30)


T = 4            # final time
num_steps = 40     # number of time steps
dt = T / num_steps # time step size


N = 15 # resolution
brain_radius = 0.1  # brain circle Radius
ventricle_radius = brain_radius/3
aqueduct_width = brain_radius/5

a_0 = 0.6
a_1 = 1.5
f_0 = 1.0/(60/15)
f_1 = 1.0/(60/70)
c = 7 

kappa = 1.5*1e-12    # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
E = 1500.0          # Young modulus
nu = 0.49999        # Poisson ratio

c = 2*1e-4  # storage coefficent
alpha = 0.5 # Biot-Willis coefficient (change of the bulk volume due to pore pressure change)

# Lame coefficients
lmbda = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
mu = E/(2.0*(1.0+nu))

mmHg2Pa = 133.32
material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K 
material_parameter["lambda"] = lmbda
material_parameter["mu"] = mu
material_parameter["alpha"] = alpha

# create mesh

mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius,
                                   aqueduct_width, N))

ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R*0.99)",
                              R = brain_radius)
skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.99 )",
                          R = brain_radius)

p_obs = Expression("""c*mmHg2Pa*(a_0 * sin(2 *pi * f_0*t) + a_1 *sin(2*pi*f_1*t))
                   *( sqrt( x[0]*x[0] + x[1]*x[1] ) - R_ventricle ) """,
                    a_0=a_0, a_1=a_0, f_0=f_0, f_1=f_1, t = 0,c=c,
                    mmHg2Pa=mmHg2Pa, R_ventricle=ventricle_radius,
                    degree = 2)

p_D_skull = p_obs
p_D_ventricle = p_obs

u_D_skull = Constant([0.0, 0.0])

probe_points = [
                Point(ventricle_radius + 0.2*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.4*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.6*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.8*(brain_radius - ventricle_radius), 0)
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 20)
slice_points = [Point(x, 0.0) for x in x_coords]

In [ ]:
control_space = FunctionSpace(mesh, "CG", 1)
ctrls = [Function(control_space, name="control") for i in range(num_steps)]
control = [Control(c) for c in ctrls]
results = solve_biot(mesh, material_parameter, T, num_steps, 
                     skull, ventricle, g=ctrls,
                     p_D_skull=p_D_skull,
                     p_D_ventricle=p_D_ventricle,
                     u_D_skull=u_D_skull,
                     probe_points=probe_points,
                     slice_points=slice_points,
                     p_target=p_obs)

rf = ReducedFunctional(results["J"], control)
problem = MinimizationProblem(rf)
parameters = {'maximum_iterations': 20}
solver = IPOPTSolver(problem, parameters=parameters)
opt_ctrls = solver.solve()
#opt_ctrls = minimize(rf, options={"maxiter": 50})


In [ ]:
# Taylor convergence Test
if False:
    h = [Function(control_space, name="control") for i in range(num_steps)]
    for func in h:
        func.vector()[:] = 0.1
    conv_rate = taylor_test(rf, ctrls, h)
#Running Taylor test
#Computed residuals: [6.377886479026454e-10, 1.5944142861939862e-10, 3.9873477688272894e-11, 9.996245970852645e-12]
#Computed convergence rates: [2.0000518769557316, 1.9995251970832304, 1.9959711335246424]

In [ ]:
xdmf_file = XDMFFile("../results/f_opt_distr_ctrl_biot.xdmf")
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False

for n in range(num_steps):
    opt_ctrls[n].rename("f","control")
    xdmf_file.write(opt_ctrls[n], n*T / num_steps)
    
results_opt = solve_biot(mesh, material_parameter, T, num_steps, 
                         skull, ventricle, g=opt_ctrls,
                         p_D_skull=p_D_skull,
                         p_D_ventricle=p_D_ventricle,
                         u_D_skull=u_D_skull,
                         probe_points=probe_points,
                         slice_points=slice_points,
                         p_target=p_obs)

p_init = results["p_fluid_slice"]
p_opt = results_opt["p_fluid_slice"]
p_target = results["p_target_slice"]
f_opt = results_opt["f_slice"]

In [ ]:
num_points = len(probe_points)
for i in range(num_points):
    fig,ax=plt.subplots(figsize=(8,8))
    plt.plot(np.linspace(0,T, num_steps),
             results["probe_point_p_target"][:,i], lw=2,
             label="observed pressure"
            )
    plt.plot(np.linspace(0,T, num_steps), 
             results["probe_point_p_fluid"][:,i], 
             label="initial pressure"
            )

    plt.plot(np.linspace(0,T, num_steps), 
             results_opt["probe_point_p_fluid"][:,i], 
             "*", markersize=10, label="optimized pressure"
            )
    plt.xlabel("t in s")
    plt.ylabel("p in Pa")
    plt.legend(loc="lower left")
    ax.twinx()

    plt.plot(np.linspace(0,T, num_steps), 
             results_opt["probe_point_f"][:,i], 
             "--", c="red",lw=5, alpha = 0.4,
             label="optimized force g")
    
    plt.ylabel("f")
    plt.grid()
    plt.legend(loc="upper right")
    plt.title("$P_{}$".format(i))

In [ ]:
times = [ 1, 2, 3, 4, 5]

idx = []
for i in range(num_steps):
    if np.isclose((i +1)*dt, times).any():
        idx.append(i)
        

for i in idx:
    fig,ax=plt.subplots(figsize=(8,8))
    plt.title("t = {}".format((i+1)*dt))
    plt.plot(x_coords, p_target[i,:], 
             label="p_target")
    plt.plot(x_coords, p_init[i,:], 
             label="p_init")
    plt.plot(x_coords, p_opt[i,:], 
             "*",label="p_opt", markersize=10)
    
    plt.legend(loc="upper left")
    plt.xlabel("x in m")
    plt.ylabel("p in Pa")
    ax.twinx()
    plt.plot(x_coords, 
             f_opt[i,:], 
             "--", c="red",
             label="optimized force")
    
    plt.ylabel("g")
    plt.grid()
    plt.legend(loc="upper center")


In [ ]:
f_max = np.max([f_opt.vector().max() for f_opt in opt_ctrls])
f_min = np.min([f_opt.vector().min() for f_opt in opt_ctrls])

for i in idx:
    plt.figure()
    c = plot(opt_ctrls[i],vmin=f_min, vmax=f_max)
    plt.colorbar(c)
    plt.title("t = {}".format((i+1)*dt))



In [ ]:
plt.figure(figsize=(10,19))
cont = plot(mesh)
for i, point in enumerate(probe_points):
    plt.scatter(point.array()[0], point.array()[1],
                marker=".", s= 200)
    plt.annotate( "$P_{}$".format(i), point.array()[0:2], size=20)

In [ ]:
plot(results["u"])